In [9]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K

In [16]:
#dir = './sequence/*'
dir = './p_sequence/*'

In [17]:
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [18]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [19]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 100

In [22]:
len(x_train)

756

In [27]:
from keras.optimizers import Adam
from keras.layers import Input, LSTM, RepeatVector, Lambda
from keras.models import Model

def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0006))

def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=756, verbose=1)#, validation_data=val_generator(x_val))

'''
- epochs 467 - 0.1900
- epochs 100 해서 확인해보면 loss 가 너무 들쑥날쑥한 걸 확인할 수 있음 1.22
- epochs 500 0.2346

- 데이터 셋 변경 epochs 500, loss - 0.0572, lr-0.001
  lr-0.0006 0.0547
'''


Epoch 1/500
756/756 [==============================] - 10s 13ms/step - loss: 52.6284
Epoch 2/500
756/756 [==============================] - 6s 8ms/step - loss: 37.0736
Epoch 3/500
756/756 [==============================] - 6s 9ms/step - loss: 30.3203
Epoch 4/500
756/756 [==============================] - 6s 8ms/step - loss: 27.1553
Epoch 5/500
756/756 [==============================] - 6s 8ms/step - loss: 25.5362
Epoch 6/500
756/756 [==============================] - 6s 8ms/step - loss: 24.0204
Epoch 7/500
756/756 [==============================] - 6s 8ms/step - loss: 22.5114
Epoch 8/500
756/756 [==============================] - 6s 8ms/step - loss: 21.2432
Epoch 9/500
756/756 [==============================] - 6s 8ms/step - loss: 21.0304
Epoch 10/500
756/756 [==============================] - 6s 8ms/step - loss: 18.9596
Epoch 11/500
756/756 [==============================] - 6s 8ms/step - loss: 18.8432
Epoch 12/500
756/756 [==============================] - 6s 8ms/step - loss: 18.9515

756/756 [==============================] - 7s 9ms/step - loss: 1.0037
Epoch 100/500
756/756 [==============================] - 6s 9ms/step - loss: 0.9118
Epoch 101/500
756/756 [==============================] - 6s 8ms/step - loss: 0.8644
Epoch 102/500
756/756 [==============================] - 6s 8ms/step - loss: 1.0232
Epoch 103/500
756/756 [==============================] - 6s 8ms/step - loss: 0.8093
Epoch 104/500
756/756 [==============================] - 6s 8ms/step - loss: 0.7260
Epoch 105/500
756/756 [==============================] - 6s 8ms/step - loss: 0.8848
Epoch 106/500
756/756 [==============================] - 6s 8ms/step - loss: 0.9264
Epoch 107/500
756/756 [==============================] - 6s 8ms/step - loss: 0.7697
Epoch 108/500
756/756 [==============================] - 6s 9ms/step - loss: 0.8606
Epoch 109/500
756/756 [==============================] - 7s 9ms/step - loss: 0.9384
Epoch 110/500
756/756 [==============================] - 7s 9ms/step - loss: 0.7871
Epoch 

756/756 [==============================] - 6s 8ms/step - loss: 0.2448
Epoch 197/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2928
Epoch 198/500
756/756 [==============================] - 6s 9ms/step - loss: 0.2309
Epoch 199/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2024
Epoch 200/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2204
Epoch 201/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2388
Epoch 202/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2394
Epoch 203/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2832
Epoch 204/500
756/756 [==============================] - 6s 8ms/step - loss: 0.2242
Epoch 205/500
756/756 [==============================] - 6s 8ms/step - loss: 0.1812
Epoch 206/500
756/756 [==============================] - 6s 8ms/step - loss: 0.1883
Epoch 207/500
756/756 [==============================] - 6s 8ms/step - loss: 0.3660
Epoch 

756/756 [==============================] - 6s 8ms/step - loss: 0.1377
Epoch 294/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0934
Epoch 295/500
756/756 [==============================] - 6s 9ms/step - loss: 0.0992
Epoch 296/500
756/756 [==============================] - 6s 9ms/step - loss: 0.1201
Epoch 297/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1296
Epoch 298/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1144
Epoch 299/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1038
Epoch 300/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1052
Epoch 301/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1194
Epoch 302/500
756/756 [==============================] - 7s 10ms/step - loss: 0.1110
Epoch 303/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1359
Epoch 304/500
756/756 [==============================] - 7s 9ms/step - loss: 0.1099
Epoch

756/756 [==============================] - 6s 8ms/step - loss: 0.0659
Epoch 391/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0732
Epoch 392/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0914
Epoch 393/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0861
Epoch 394/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0729
Epoch 395/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0831
Epoch 396/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0870
Epoch 397/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0858
Epoch 398/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0731
Epoch 399/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0744
Epoch 400/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0925
Epoch 401/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0845
Epoch 

756/756 [==============================] - 7s 9ms/step - loss: 0.0454
Epoch 488/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0544
Epoch 489/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0604
Epoch 490/500
756/756 [==============================] - 6s 9ms/step - loss: 0.0664
Epoch 491/500
756/756 [==============================] - 6s 9ms/step - loss: 0.0644
Epoch 492/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0576
Epoch 493/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0493
Epoch 494/500
756/756 [==============================] - 6s 9ms/step - loss: 0.0494
Epoch 495/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0643
Epoch 496/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0693
Epoch 497/500
756/756 [==============================] - 7s 9ms/step - loss: 0.0592
Epoch 498/500
756/756 [==============================] - 6s 8ms/step - loss: 0.0454
Epoch 

'\n- epochs 467 - 0.1900\n- epochs 100 해서 확인해보면 loss 가 너무 들쑥날쑥한 걸 확인할 수 있음 1.22\n- epochs 500 0.2346\n\n- 데이터 셋 변경 epochs 500, loss - 0.0572, lr-0.001\n  lr-0.003\n'

In [28]:
encoder = Model(inputs, encoded)

In [29]:
lstm_autoencoder.summary()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, None, 3)      0                                            
__________________________________________________________________________________________________
lstm_41 (LSTM)                  (None, None, 128)    67584       input_11[0][0]                   
__________________________________________________________________________________________________
lstm_42 (LSTM)                  (None, 64)           49408       lstm_41[0][0]                    
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, None, 64)     0           lstm_42[0][0]                    
                                                                 input_11[0][0]            

In [30]:
encoder.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, None, 3)           0         
_________________________________________________________________
lstm_41 (LSTM)               (None, None, 128)         67584     
_________________________________________________________________
lstm_42 (LSTM)               (None, 64)                49408     
Total params: 116,992
Trainable params: 116,992
Non-trainable params: 0
_________________________________________________________________


In [33]:
lstm_autoencoder.summary()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, None, 3)      0                                            
__________________________________________________________________________________________________
lstm_41 (LSTM)                  (None, None, 128)    67584       input_11[0][0]                   
__________________________________________________________________________________________________
lstm_42 (LSTM)                  (None, 64)           49408       lstm_41[0][0]                    
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, None, 64)     0           lstm_42[0][0]                    
                                                                 input_11[0][0]            

In [42]:
lstm_autoencoder.predict(x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1]))

array([[[ 8.931337 ,  7.4579573, 31.943764 ],
        [ 8.431695 ,  7.737891 ,  8.375352 ],
        [ 7.3948007,  5.0614414, 13.785835 ],
        [ 7.925072 ,  5.933628 , 14.485416 ],
        [ 2.0842404,  1.931783 ,  8.705457 ],
        [ 2.2869372,  3.9760273,  7.5900183],
        [ 7.5529623,  5.3252344, 26.848215 ],
        [ 2.7739074,  3.8996158,  8.020057 ],
        [ 1.5346651,  4.882212 , 11.469937 ]]], dtype=float32)

In [45]:
x_test[2]

array([[ 4.  ,  2.  , 10.06],
       [ 2.  ,  1.  ,  8.79],
       [ 2.  ,  5.  ,  6.86],
       [ 1.  ,  3.  , 11.79],
       [ 3.  ,  6.  ,  7.86]])

In [49]:
x_test[4]

array([[ 5.  ,  3.  , 23.54],
       [ 5.  ,  4.  , 42.39],
       [ 3.  ,  1.  ,  7.22],
       [ 4.  ,  2.  , 12.22],
       [ 1.  ,  2.  ,  5.7 ]])

In [47]:
x_test[3]

array([[ 1.  ,  2.  ,  3.3 ],
       [ 1.  ,  3.  ,  6.3 ],
       [ 2.  ,  4.  , 12.72],
       [ 3.  ,  5.  , 23.87],
       [ 4.  ,  5.  , 41.95]])

In [76]:
test = x_test[0].reshape(1, 9, 3)
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [161]:
latent_vector = np.array(latent_vector)

[-0.0,
 -0.00020211579,
 0.0,
 0.0,
 0.74070954,
 -0.029288653,
 -0.74629986,
 0.0,
 0.12747759,
 -0.0014260766,
 -0.0,
 0.10494814,
 0.2187337,
 -0.18827741,
 0.040405355,
 -0.5645764,
 0.0022893408,
 0.0,
 0.0,
 -0.2676836,
 -0.7395759,
 0.2642008,
 -0.41065842,
 0.19330677,
 -0.3758831,
 -0.21791713,
 0.055199105,
 -0.25927162,
 0.23109113,
 0.8425839,
 -0.03407281,
 -0.8132615,
 0.0,
 0.49131706,
 0.16162786,
 0.35315764,
 -0.0,
 -0.7594612,
 -0.6364968,
 0.0,
 0.009271684,
 0.0,
 0.30094486,
 0.0,
 -0.0027280087,
 -0.25117594,
 0.0,
 0.19940257,
 0.32026434,
 0.5843708,
 0.032265846,
 -0.0,
 -0.0140618635,
 -0.0,
 0.0,
 0.0,
 -0.0,
 0.254268,
 -0.0,
 0.0,
 -0.0,
 0.19169566,
 0.0,
 0.7144473]

In [144]:
def printMatrix(matrix):
    length = int(max(max(matrix[:, 0]), max(matrix[:, 1])))
    adj = [[0 for i in range(length)] for i in range(length)]
    
    for m in matrix:
        (i, j, w) = m
        i = int(i) -1
        j = int(j) -1
        adj[i][j] = adj[j][i] = w
    for i in range(length):
        for j in range(length):
            print(str(adj[i][j]), end=',')
        print()
    print("===============")

In [213]:
# sequence to matrix
for index, x in enumerate(x_test):
    print("#", str(index))
    printMatrix(x)
    

# 0
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0,0,0,
0,0,18.05,0,0,0,0,32.09,0,
0,0,0,20.05,0,0,0,0,20.27,
0,0,0,0,0,32.09,0,0,9.01,
0,0,0,0,0,0,20.27,9.01,0,
# 1
0,8.79,11.79,0,0,0,
8.79,0,0,10.06,6.86,0,
11.79,0,0,0,0,7.86,
0,10.06,0,0,0,0,
0,6.86,0,0,0,0,
0,0,7.86,0,0,0,
# 2
0,3.3,6.3,0,0,
3.3,0,0,12.72,0,
6.3,0,0,0,23.87,
0,12.72,0,0,41.95,
0,0,23.87,41.95,0,
# 3
0,5.7,7.22,0,0,
5.7,0,0,12.22,0,
7.22,0,0,0,23.54,
0,12.22,0,0,42.39,
0,0,23.54,42.39,0,
# 4
0,12.52,17.52,28.77,0,0,0,0,0,
12.52,0,0,0,0,0,0,0,0,
17.52,0,0,0,2.53,4.53,0,0,0,
28.77,0,0,0,0,0,10.86,0,0,
0,0,2.53,0,0,0,0,0,0,
0,0,4.53,0,0,0,0,9.01,0,
0,0,0,10.86,0,0,0,0,10.01,
0,0,0,0,0,9.01,0,0,20.01,
0,0,0,0,0,0,10.01,20.01,0,
# 5
0,4.27,9.29,0,0,
4.27,0,0,12.29,0,
9.29,0,0,0,17.29,
0,12.29,0,0,3.95,
0,0,17.29,3.95,0,
# 6
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0

In [171]:
standard = latent_vector[0]

In [223]:
import copy
dist = []
left = 100000
right = -10000
for index, v in enumerate(latent_vector[1:]):
    d = round(euclidean_distance(standard, v), 8)
    dist.append(d)
    if d > right:
        right= d
    if left > d:
        left = d
dist1 = copy.deepcopy(dist)
dist1.sort()
#print(dist, dist1)
for index, d in enumerate(dist1):
    print( (index+1, dist.index(d)+1), end='  ')

(1, 2)  (2, 38)  (3, 53)  (4, 5)  (5, 15)  (6, 44)  (7, 48)  (8, 21)  (9, 12)  (10, 43)  (11, 9)  (12, 18)  (13, 31)  (14, 59)  (15, 58)  (16, 4)  (17, 41)  (18, 27)  (19, 29)  (20, 33)  (21, 6)  (22, 3)  (23, 51)  (24, 35)  (25, 54)  (26, 55)  (27, 26)  (28, 62)  (29, 14)  (30, 46)  (31, 11)  (32, 45)  (33, 7)  (34, 50)  (35, 57)  (36, 8)  (37, 63)  (38, 60)  (39, 24)  (40, 52)  (41, 20)  (42, 13)  (43, 37)  (44, 23)  (45, 10)  (46, 1)  (47, 61)  (48, 42)  (49, 56)  (50, 34)  (51, 16)  (52, 32)  (53, 40)  (54, 39)  (55, 19)  (56, 17)  (57, 22)  (58, 49)  (59, 30)  (60, 36)  (61, 47)  (62, 28)  (63, 25)  

In [168]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum(a-b) ** 2)

In [221]:
len(x_test)

64